<a href="https://colab.research.google.com/github/WendyValdes/Virtual-assistant/blob/main/Virt_ass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalar bibliotecas

In [4]:
!pip install SpeechRecognition gTTS wikipedia

Função para gravar áudio no Colab

In [5]:
from IPython.display import HTML, Audio, display
import speech_recognition as sr
from gtts import gTTS
import wikipedia
import webbrowser
import base64

wikipedia.set_lang("pt")

In [6]:
# ---- Registrar função de gravação no Colab (ESSA PARTE É ESSENCIAL) ----
from google.colab import output

def salvar_audio(b64_audio):
    audio_data = base64.b64decode(b64_audio.split(',')[1])
    with open("comando.wav", "wb") as f:
        f.write(audio_data)

output.register_callback("notebook.record_audio", salvar_audio)

Assistente Virtual

In [26]:
# ---- Criar botão de gravação REAL ----
display(HTML("""
<script>
var record = () => {
  return new Promise(async (resolve) => {
    const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
    const mediaRecorder = new MediaRecorder(stream);
    let chunks = [];

    mediaRecorder.ondataavailable = e => chunks.push(e.data);
    mediaRecorder.start();

    await new Promise(r => setTimeout(r, 5000)); // grava 5 segundos
    mediaRecorder.stop();

    mediaRecorder.onstop = () => {
      const blob = new Blob(chunks, { type: 'audio/wav' });
      const reader = new FileReader();
      reader.readAsDataURL(blob);
      reader.onloadend = () => resolve(reader.result);
    };
  });
}

function gravar() {
  record().then(data => {
    google.colab.kernel.invokeFunction('notebook.record_audio', [data], {});
  });
}
</script>

<button onclick="gravar()">🎤 Gravar 5 segundos</button>
"""))

print("Clique no botão acima e fale seu comando!")


Clique no botão acima e fale seu comando!


In [27]:
from IPython.display import Audio, display

display(Audio("comando.wav", autoplay=True))

In [28]:
import subprocess

# Converter o áudio para WAV padrão compatível com SpeechRecognition
!ffmpeg -y -i comando.wav comando_convertido.wav

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Executar comando por voz

In [30]:
# -------- Reconhecer fala --------
import speech_recognition as sr

r = sr.Recognizer()

with sr.AudioFile("comando_convertido.wav") as source:
    audio = r.record(source)

try:
    comando = r.recognize_google(audio, language="pt-BR")
    print(f"Você disse: {comando}")
    comando = comando.lower()
except Exception as e:
    print("Erro ao reconhecer:", e)
    comando = ""

# -------- Responder por áudio --------
def falar(texto):
    print(f"Assistente: {texto}")
    tts = gTTS(texto, lang="pt")
    tts.save("resposta.mp3")
    display(Audio("resposta.mp3", autoplay=True))

# -------- Comandos --------
if "youtube" in comando:
    falar("Abrindo YouTube")
    webbrowser.open("https://www.youtube.com")

elif "wikipédia" in comando or "wikipedia" in comando:
    falar("Abrindo wikipédia.")
    abrir_link_colab("https://pt.wikipedia.org/wiki/Wikip%C3%A9dia:P%C3%A1gina_principal")

elif "farmácia" in comando:
    falar("Abrindo farmácias próximas")
    webbrowser.open("https://www.google.com/maps/search/farmacia+perto+de+mim")

else:
    falar("Diga: YouTube, Wikipédia ou Farmácia.")


Você disse: abrindo Wikipédia
Assistente: Abrindo wikipédia.
